<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/Macd_Tilson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
# Gerekli paketleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener backtesting
# 'StockIndicators.py' dosyasını indirin
if os.path.exists('StockIndicators.py'):
    os.remove('StockIndicators.py')
!wget https://raw.githubusercontent.com/Onuragdaci/Stock_Indicators/main/StockIndicators.py

import os
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy
from tradingview_screener import get_all_symbols
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import StockIndicators as SI

# Initialize TradingView datafeed
tv = TvDatafeed()

# Get all BIST symbols
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

def calc_dema(close, period):
    ema1 = close.ewm(span=period, adjust=False).mean()
    ema2 = ema1.ewm(span=period, adjust=False).mean()
    dema = 2 * ema1 - ema2
    return dema

def macd_dema_squeeze_momentum(data, sma=12, lma=26, tsp=9):
    close = data['Close']
    # DEMA Calculations
    DEMAfast = calc_dema(close, sma)
    DEMAslow = calc_dema(close, lma)

    LigneMACDZeroLag = DEMAfast - DEMAslow

    MMEsignala = LigneMACDZeroLag.ewm(span=tsp, adjust=False).mean()
    MMEsignalb = MMEsignala.ewm(span=tsp, adjust=False).mean()
    Lignesignal = 2 * MMEsignala - MMEsignalb

    MACDZeroLag = LigneMACDZeroLag - Lignesignal

    return MACDZeroLag, Lignesignal

def ema(series, length):
    return series.ewm(span=length, adjust=False).mean()

def TillsonT3(data, Length=14, vf=0.7, app=2):
    df = data.copy()
    ema_first_input = (df['High'] + df['Low'] + 2 * df['Close']) / 4
    e1 = ema(ema_first_input, Length)
    e2 = ema(e1, Length)
    e3 = ema(e2, Length)
    e4 = ema(e3, Length)
    e5 = ema(e4, Length)
    e6 = ema(e5, Length)

    c1 = -1 * vf * vf * vf
    c2 = 3 * vf * vf + 3 * vf * vf * vf
    c3 = -6 * vf * vf - 3 * vf - 3 * vf * vf * vf
    c4 = 1 + 3 * vf + vf * vf * vf + 3 * vf * vf
    df['T3'] = c1 * e6 + c2 * e5 + c3 * e4 + c4 * e3
    df['Entry_Tillson'] = df['T3'] > df['T3'].shift(1)
    df['Exit_Tillson'] = df['T3'] < df['T3'].shift(1)
    return df

# Process each stock
for stock in Hisseler:
    try:
        # Fetch historical data
        data = tv.get_hist(symbol=stock, exchange='BIST', interval=Interval.in_daily, n_bars=100)
        if data is None or data.empty:
            print(f"No data available for {stock}")
            continue

        # Preprocess data
        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        data = data.reset_index()

        # Calculate indicators
        Tillson = TillsonT3(data, 7, 0.7, 2)
        MACDZeroLag, Lignesignal = macd_dema_squeeze_momentum(data)

        # Combine with data
        data['MACDZeroLag'] = MACDZeroLag
        data['Lignesignal'] = Lignesignal
        data['MACD_Entry'] = MACDZeroLag > Lignesignal
        data['MACD_Exit'] = MACDZeroLag < Lignesignal

        # Check buy condition for the last candle for MACD and last 2 candles for Tillson
        macd_buy_condition = data['MACD_Entry'].iloc[-1]
        tillson_buy_condition = any(
            Tillson['Entry_Tillson'].iloc[-i] and Tillson['Exit_Tillson'].iloc[-i-1]
            for i in range(1, 3)
        )

        if macd_buy_condition and tillson_buy_condition:
            print(f"Buy signal for {stock}")

    except Exception as e:
        print(f"Error processing {stock}: {e}")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-1nfhthcx
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-1nfhthcx
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
--2024-09-02 00:28:03--  https://raw.githubusercontent.com/Onuragdaci/Stock_Indicators/main/StockIndicators.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70277 (69K) [text/plain]
Saving to: ‘StockIndicators.py’

StockIndicators.py  100%[===================>]  68.63K  --.-KB/s    in 0.02s   

2024-09-02 00:28:03 (4.31 MB/s) - ‘StockIndicators.py’ saved [70277/70277]



Error processing AHSGY: single positional indexer is out-of-bounds
Buy signal for AKFGY
Buy signal for ARDYZ
Buy signal for ASTOR
Buy signal for BOSSA
Buy signal for CRFSA
Buy signal for DCTTR
Buy signal for EDIP
Buy signal for EFORC
Buy signal for ENERY
Buy signal for FENER
Buy signal for IHAAS
Buy signal for INTEK
Buy signal for INTEM
Buy signal for KLKIM
Buy signal for KUYAS
Buy signal for LINK
Buy signal for MMCAS
Buy signal for PGSUS
Buy signal for PINSU
Buy signal for PNSUT
Buy signal for POLHO
Buy signal for SISE
Buy signal for SODSN
Buy signal for TAVHL
Buy signal for VAKKO
